In [1]:
import anthropic
import base64
import pandas as pd
import base64
import re
import os
import numpy as np
import requests
import ollama

pd.options.mode.chained_assignment = None

In [2]:
macronutrients_instruction = '''Examine the provided meal image to analyze and estimate its nutritional content accurately. Focus on determining the amounts of simple sugars (like industrial sugar and honey), 
complex sugars (such as starch and whole grains), proteins, fats, and dietary fibers (found in fruits and vegetables), all in grams. Also estimate the total weight of the meal in grams.
To assist in accurately gauging the scale of the meal, a 1 Swiss Franc coin, which has a diameter of 23.22 mm, may be present in the picture. 
Use the size of this coin as a reference to estimate the size of the meal and the amounts of the nutrients more precisely. 
Provide your assessment of each nutritional component in grams. All estimates should be given as a single whole number. If there is no coin in the picture or the meal is covered partially, estimate anyways.
Format your response as follows:
- Simple sugars (g): 
- Complex sugars (g): 
- Proteins (g): 
- Fats (g): 
- Dietary fibers (g): 
- Weight (g): 
- Explanation: 

Example response:
Simple sugars (g): 40
Complex sugars (g): 60
Proteins (g): 25
Fats (g): 30
Dietary fibers (g): 5 
Weight (g): 750
Explanation: The pizza and cola meal, with its refined crust and toppings, is rich in carbs, fats, and proteins. The cola boosts the meal's simple sugars. 
The 1 Swiss Franc coin helps estimate the pizza at 30 cm diameter and the cola at 330 ml, indicating a significant blood sugar impact.'''

In [3]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def parse_nutritional_info(text):
    pattern = r'(Simple sugars \(g\)|Complex sugars \(g\)|Proteins \(g\)|Fats \(g\)|Dietary fibers \(g\)|Weight \(g\)):\s*(\d+)'
    matches = re.findall(pattern, text)
    nutritional_info = {match[0]: int(match[1]) for match in matches}
    simple_sugars = nutritional_info.get('Simple sugars (g)', 0)
    complex_sugars = nutritional_info.get('Complex sugars (g)', 0)
    proteins = nutritional_info.get('Proteins (g)', 0)
    fats = nutritional_info.get('Fats (g)', 0)
    dietary_fibers = nutritional_info.get('Dietary fibers (g)', 0)
    weight = nutritional_info.get('Weight (g)', 0)
    return simple_sugars, complex_sugars, proteins, fats, dietary_fibers, weight

In [13]:
claude_client = anthropic.Anthropic(api_key="sk-WOih6ZqdppfFCsptpJcFsSpWJ8u-9Cuy_-6N-inqcST3BlbkFJnff9SwAVU-YXkjDVdoOA1p9LAgfhqzhgsTRWZimmQA")

api_key = "sk-WOih6ZqdppfFCsptpJcFsSpWJ8u-9Cuy_-6N-inqcST3BlbkFJnff9SwAVU-YXkjDVdoOA1p9LAgfhqzhgsTRWZimmQA"
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

In [14]:
def claude_api_call(base64_image):
    response = claude_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",  
                            "data": base64_image,
                        },
                    },
                    {
                        "type": "text",
                        "text": macronutrients_instruction
                    }
                ],
            }
        ],
    )
    message = response.content[0].text
    return message

def openai_api_call(base64_image):
    payload = {
    "model": "gpt-4o",
    "messages": [
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": macronutrients_instruction
            },
            {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
            }
        ]
        }
    ],
    "max_tokens": 300
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    message = response.json()['choices'][0]['message']['content']
    return message

# make sure to install ollama and llama3.2-vision
def ollama_call(image_path):
    res = ollama.chat(
        model="llama3.2-vision",
        messages=[
            {
                'role': 'user',
                'content': macronutrients_instruction,
                'images': [image_path]
            }
        ]
    )
    message = res['message']['content']
    return message

In [15]:
for model in ['gpt4o', 'ollama']:
    print(f"Processing {model}")
    for patient in ['001', '002', '004', '006', '007', '008']:
        print(f"Processing patient {patient}")
        food_data_path = f"diabetes_subset_pictures-glucose-food-insulin/{patient}/food.csv"
        food_data = pd.read_csv(food_data_path)
        food_data['simple_sugars'] = 0.0
        food_data['complex_sugars'] = 0.0 
        food_data['proteins'] = 0.0
        food_data['fats'] = 0.0
        food_data['dietary_fibers'] = 0.0
        food_data['weight'] = 0.0
        food_data['message'] = ''
        
        for i, row in food_data.iterrows():
            image_path = f"diabetes_subset_pictures-glucose-food-insulin/{patient}/food_pictures/{row['picture']}"
            base64_image = encode_image(image_path)
            parsed_info = (0.0, 0.0, 0.0, 0.0, 0.0)
            
            while np.sum(parsed_info) == 0:
                if model == 'claude':
                    message = claude_api_call(base64_image)
                elif model == 'openai':
                    message = openai_api_call(base64_image)
                    print(message)
                elif model == 'ollama':
                    message = ollama_call(image_path)
                parsed_info = parse_nutritional_info(message)
                if np.sum(parsed_info) > 0:
                    food_data.loc[i, 'simple_sugars'] = parsed_info[0]
                    food_data.loc[i, 'complex_sugars'] = parsed_info[1]
                    food_data.loc[i, 'proteins'] = parsed_info[2]
                    food_data.loc[i, 'fats'] = parsed_info[3]
                    food_data.loc[i, 'dietary_fibers'] = parsed_info[4]
                    food_data.loc[i, 'weight'] = parsed_info[5]
                    food_data.loc[i, 'message'] = str(message)
        #food_data.to_csv(f'food_data/{model}/{patient}.csv', index=False)

Processing gpt4o
Processing patient 001


NameError: name 'message' is not defined

In [16]:
res = openai_api_call(base64_image)

In [17]:
res.json()

{'id': 'chatcmpl-AaPqTFyMDaoc4qUWskQQOy1jYyz8m',
 'object': 'chat.completion',
 'created': 1733242521,
 'model': 'gpt-4o-2024-08-06',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "Simple sugars (g): 2  \nComplex sugars (g): 5  \nProteins (g): 3  \nFats (g): 4  \nDietary fibers (g): 10  \nWeight (g): 150  \nExplanation: The image shows a bowl of chopped lettuce or cabbage. The presence of a 1 Swiss Franc coin aids in estimating the bowl's size, suggesting a moderate portion. The dish is primarily composed of leafy greens, which are low in simple and complex sugars and moderate in dietary fiber. The minimal dressing or seasoning could contribute to small amounts of fats and simple sugars.",
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 1452,
  'completion_tokens': 122,
  'total_tokens': 1574,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_